This example uses the default GLiNER model to perform named entity recognition.



In [15]:
from pydantic import BaseModel, field_validator
from typing import List

class Entity(BaseModel):
    entity: str
    types: str
    
    @field_validator('entity')
    @classmethod
    def clean_entity(cls, v):
        # Take only the text before the comma if there is one
        if isinstance(v, str) and "," in v:
            return v.split(",")[0].strip()
        return v

class Result(BaseModel):
    text: str
    entities: List[Entity]

In [ ]:
import json
from pathlib import Path

test_set_path = Path("results.jsonl")

results = []

with open(test_set_path, "r") as f:
    for line in f:
        raw_data = json.loads(line)
        try:
            result = Result(**raw_data)
            results.append(result)
        except Exception as e:
            print(f"Error parsing line: {e}")
            print(f"Problematic data: {raw_data}")

In [21]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_large-v2.1")

labels = ["Interval", "Money", "Start Date", "Phone", "Address", "Person", "Faculty"]

for result in results:
    text = result.text
    print(text)
    # Use the correct API for text input - GLiNER expects text directly
    entities = model.extract_entities(text, labels, threshold=0.8)
    print(entities)


Fetching 4 files: 100%|██████████| 4/4 [00:00<?, ?it/s]


Dear University of Manitoba Development Office,
I am delighted to be supporting your cause through a one-time gift of $20,000.
I would like this donation to go towards the Computer Science Department's Fellowship Program. Please let me know if this is feasible and how we can make it happen. 
Thank you for your time and consideration.
Sincerely,
Michael Roberts.


AttributeError: 'GLiNER' object has no attribute 'extract_entities'